In [1]:
# Quick and dirty hack because my PyCharm interpreter is not playing nice
ewouts_broken_PyCharm = True
if ewouts_broken_PyCharm:
    import sys
    sys.path.append('C:\\Users\\Ewout\\Documents\\python_venv\\Py310')
    sys.path.append('C:\\Users\\Ewout\\Documents\\python_venv\\Py310\\lib\\site-packages')

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import sys
import os
import itertools
import pandas as pd
import pickle
from numpy import random
from time import sleep

### Scaper
https://www.msc.com/en/search-a-schedule

In [3]:
# Create list of origin and destination ports
o_ports = ['BRFOR', 'BRSSZ', 'BRPEC', 'BRSSA', 'BRPNG', 'BRRIO', 'BRRIG', 'BRITJ', 'BRNAT', 'COCTG', 'COSMR', 'COTRB', 'SRPBM', 'CWWIL', 'UYMVD', 'ARBUE', 'CLCNL', 'CLSAI', 'CLVAP', 'PEPAI', 'PECLL', 'ECGYE', 'ECPBO', 'ECPSJ', 'VNVUT', 'VNCMT']
d_ports = ['NLRTM', 'BEANR', 'NLVLI', 'BEZEE', 'NLAMS']

# Make list with all combinations
od_ports = list(itertools.product(o_ports, d_ports))
print(f"{len(od_ports)} combinations of ports")

# Define the base URL to search
base_url = "https://www.msc.com/en/search-a-schedule"

130 combinations of ports


In [4]:
headless = False

# Instantiate options
opts = Options()
#opts.binary_location = "/usr/bin/google-chrome"
opts.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"
if headless:
    opts.headless = True
opts.add_argument('--remote-debugging-port=9222')
opts.add_argument('--disable-gpu')
opts.add_argument("--window-size=1920,1440")

# Some options to make Chrome (hopefully) more
opts.add_argument('--disable-blink-features=AutomationControlled')
opts.add_experimental_option('useAutomationExtension', False)
opts.add_experimental_option("excludeSwitches", ["enable-automation"])

# Set the location of the webdriver
os.environ["WDM_PROGRESS_BAR"] = '0'
chrome_service = Service(ChromeDriverManager().install())

# Instantiate a webdriver
driver = webdriver.Chrome(options=opts, service=chrome_service)

In [5]:
# Get a single pair
o, d = od_ports[0]
o = "BRSSZ"
direct = True

# Load the HTML page
driver.get(base_url)

# Accept cookies
driver.implicitly_wait(3)
try:
    driver.find_element(By.ID,"onetrust-accept-btn-handler").click()
except:
    print("Cookies already accepted.")

# Fill in and select the origin and destination port
for label, port in [("from", o), ("to", d)]:
    elem = driver.find_element(By.ID, label)
    elem.send_keys(port)
    sleep(0.5) ### TODO make this a proper wait
    ActionChains(driver).send_keys(Keys.TAB).send_keys(Keys.ENTER).perform()

# Select direct routing
if direct:
    driver.find_element(By.ID, "directrouting").click()

sleep(1)
# Click search button
driver.find_element(By.CSS_SELECTOR, "button.msc-cta").click()

# Find all buttons to open
buttons = driver.find_elements(By.CSS_SELECTOR, "button.msc-search-schedule-result-details__more-button")

# Open the buttons that are displayed
for btn in buttons:
    if btn.is_displayed():
        btn.click()

# Save soup
#sleep(1) ### TODO proper wait
soup = BeautifulSoup(driver.page_source, features="html.parser")

In [6]:
soup = BeautifulSoup(driver.page_source, features="html.parser")

### Soup

In [7]:
print(len(soup.text))

17907


In [8]:
# Split the soup in different connections
connections = soup.find_all("div", {"class": "msc-search-schedule-result-details__result"})
#connections = soup.find_all("div", {"class": "msc-search-schedule-result-details__tracking"})
len(connections)

9

In [9]:
# Split each connection into different data cells
data_cells = {}
for i, connection in enumerate(connections):
    data_cells[i] = connection.find_all("div", {"class": "msc-search-schedule-result-details__timeline-data"})

In [10]:
type(data_cells[2])

bs4.element.ResultSet

### Notes
 - It looks like the first two connections are empty (9 connections in the list, 7 on the website, first 2 contain no data)
  - Departure time has multiple fields, the second/last one contains the actual time. Therefore the [1] on the end.

In [11]:
departure_time = connections[4].find_all("span", {"x-text": "getLegSequenceValue(entry, 'EstimatedDepartureTimeFormatted')"})[1].text
print(departure_time)

Sun 13th Nov 2022 01:00
